In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as pltly
import plotly.express as px
import seaborn as sns
import scipy.stats as stats
import tap
import pandas as pd
from scipy.stats import skew, kurtosis

In [4]:
pip install pycountry pycountry-convert

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Read the csv file SOC perrenials DATABASE.csv from the subfolder SOC-project
#This accesses a public github repository so all of us can run this file on our computers since the file is stored remotely
df = pd.read_csv(r"https://github.com/emmadefrang/MGT-499/raw/refs/heads/main/SOC-project/SOC%20perennials%20DATABASE.csv",
                 skiprows=1,
                 encoding = 'cp1252')
df.head()

#Print the actual total number of rows and columns in the dataset
print(df.shape)

(1605, 72)


In [6]:
link = 'https://raw.githubusercontent.com/emmadefrang/MGT-499/refs/heads/main/SOC-project/land_use_change_rank.csv'

luc_rank = pd.read_csv(link)

In [7]:
df.columns

Index(['ID', 'IDstudy', 'plotID', 'country', 'region', 'temperature_Celsiul',
       'precipitation_mm', 'climate', 'Latitud', 'Longitud', 'bedrock',
       'soil_type', 'USDA', 'TYPE_CHANGE', 'CROP_current', 'CROP_type',
       'cropAGE', 'current_land_use*', 'LUC_this_study', 'previous_land_use*',
       'years_since_luc', 'TEMPORAL_only_this study', 'timeTEMPORAL',
       'CHANGE_management', 'year_since_MANAGEMENT_CHANGE', 'soil_treatments',
       'tillageYESNO', 'tillage_type', 'fertilizerYESNO', 'fet_type',
       'rate_fertilizer_kh_ha', 'pesticides', 'org_amendYESNO',
       'org_amend_type', 'prev_man', 'SINGLE_DATA-MULTIPLE_SINGLEAMPLING',
       'DIF_DEPTH_current_point', 'CONTI_DEPTH', 'NUM_DEPTH', 'year_measure',
       'N_plots_current', 'N_sample_per_plot_current', 'N_measured_current',
       'soil_from_cm_current', 'soil_to_cm_current', 'depth_midpoint_current',
       'bulk_density_Mg_m3_current', '%clay_current', '%silt_current',
       '%sand_current', 'ph_current'

In [8]:
# Filter out rows where SOC stock is NaN for both current and previous
df = df[(~df['SOC_Mg_ha_current'].isna()) | (~df['SOC_Mg_ha_previous'].isna())]

df = df[df['CHANGE_management'] == 'NO']

print(df.head())


   ID  IDstudy  plotID country    region  temperature_Celsiul  \
0   1        1       1  Brazil  SaoPaulo                 21.0   
1   2        1       1  Brazil  SaoPaulo                 21.0   
2   3        1       1  Brazil  SaoPaulo                 21.0   
3   4        1       1  Brazil  SaoPaulo                 21.0   
4   5        1       2  Brazil  SaoPaulo                 21.0   

   precipitation_mm   climate  Latitud  Longitud  ... %clay_previous  \
0            1500.0  Tropical   -21.35   -47.066  ...           19.2   
1            1500.0  Tropical   -21.35   -47.066  ...           21.7   
2            1500.0  Tropical   -21.35   -47.066  ...           24.8   
3            1500.0  Tropical   -21.35   -47.066  ...           24.8   
4            1500.0  Tropical   -21.35   -47.066  ...           42.9   

  %silt_previous %sand_previous ph_previous SOC_g_kg_previous  \
0           10.4           70.5         NaN               NaN   
1            8.9           69.3         NaN   

In [9]:
#How many observations are we missing pH for
print(df['ph_previous'].isna().sum())
print(df['ph_current'].isna().sum())

703
584


In [10]:
#Only keep the rows where we have a value for pH for both current and previous. We will call this a new dataframe so we can test its performance 
#against the model that will neglect pH

df_ph = df[(~df['ph_previous'].isna()) & (~df['ph_current'].isna())]

print(df_ph.shape)

(260, 72)


In [11]:
# Create separate dataframes for 'current' and 'previous' data
current_df = df[['ID', 'IDstudy', 'plotID', 'country', 'region', 'climate', 'temperature_Celsiul', 'precipitation_mm', '%clay_current', '%silt_current', '%sand_current', 
                 'ph_current', 'SOC_Mg_ha_current', 'soil_from_cm_current', 'soil_to_cm_current', 'depth_midpoint_current']].copy()
previous_df = df[['ID', 'IDstudy', 'plotID', 'country', 'region', 'climate', 'temperature_Celsiul', 'precipitation_mm', '%clay_previous', '%silt_previous', '%sand_previous', 
                  'ph_previous', 'SOC_Mg_ha_previous', 'soil_from_cm_previous', 'soil_to_cm_previous', 'depth_midpoint_previous']].copy()

# Rename columns for clarity
current_df.columns = ['ID', 'IDstudy', 'plotID', 'country', 'region', 'climate', 'temperature', 'precipitation', 'clay', 'silt', 'sand', 'pH', 'SOC_Mg_ha', 'soil_from_cm', 'soil_to_cm', 'depth_midpoint']
previous_df.columns = ['ID', 'IDstudy', 'plotID', 'country', 'region', 'climate', 'temperature', 'precipitation', 'clay', 'silt', 'sand', 'pH', 'SOC_Mg_ha', 'soil_from_cm', 'soil_to_cm', 'depth_midpoint']

# Add a column to indicate current vs previous
current_df['Period'] = 'Current'
previous_df['Period'] = 'Previous'

# Concatenate the dataframes
combined_df = pd.concat([current_df, previous_df], ignore_index=True)

# Drop rows with NaN SOC_Mg_ha
combined_df.dropna(subset=['SOC_Mg_ha'], inplace=True)

# Remove the pH column
combined_df = combined_df.drop(columns=['pH'])

print(combined_df.head())

   ID  IDstudy  plotID country    region   climate  temperature  \
0   1        1       1  Brazil  SaoPaulo  Tropical         21.0   
1   2        1       1  Brazil  SaoPaulo  Tropical         21.0   
2   3        1       1  Brazil  SaoPaulo  Tropical         21.0   
3   4        1       1  Brazil  SaoPaulo  Tropical         21.0   
4   5        1       2  Brazil  SaoPaulo  Tropical         21.0   

   precipitation  clay  silt  sand  SOC_Mg_ha  soil_from_cm  soil_to_cm  \
0         1500.0  13.7   5.1  81.2      20.70           0.0        10.0   
1         1500.0  14.8   1.1  84.2      12.54          10.0        20.0   
2         1500.0  18.0   2.0  80.0      30.92          20.0        60.0   
3         1500.0  18.8   3.5  77.7      36.23          60.0       100.0   
4         1500.0  38.1  10.9  51.0      16.43           0.0        10.0   

   depth_midpoint   Period  
0             5.0  Current  
1            15.0  Current  
2            40.0  Current  
3            80.0  Current  
4

In [12]:
#How many NaN values do we have left in the dataset
print(combined_df.isna().sum())

#How long is the dataset now
combined_df.shape

#Drop all rows with NaN values
combined_df = combined_df.dropna()

#How long is the dataset now
combined_df.shape

ID                   0
IDstudy              0
plotID               0
country              0
region               0
climate              0
temperature         26
precipitation       18
clay              1217
silt              1221
sand              1305
SOC_Mg_ha            0
soil_from_cm         0
soil_to_cm           0
depth_midpoint       0
Period               0
dtype: int64


(612, 16)

In [13]:
# Define your feature set (X) and target variable (y)
features = ['temperature', 'precipitation', 'clay', 'silt', 'sand','depth_midpoint']
X = combined_df[features]
y = combined_df['SOC_Mg_ha']

In [14]:
# Encode the 'Period' variable and add interaction terms
combined_df['Period'] = combined_df['Period'].map({'Current': 1, 'Previous': 0})
for feature in features:
    combined_df[f'{feature}_x_Period'] = combined_df[feature] * combined_df['Period']

# Update feature set to include interaction terms
interaction_features = features + [f'{feature}_x_Period' for feature in features]
X = combined_df[interaction_features]


In [15]:
import statsmodels.api as sm

# Add a constant for the intercept
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              SOC_Mg_ha   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     71.72
Date:                Thu, 14 Nov 2024   Prob (F-statistic):          2.14e-107
Time:                        13:38:54   Log-Likelihood:                -3027.9
No. Observations:                 612   AIC:                             6082.
Df Residuals:                     599   BIC:                             6139.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

Our R2 value is relatively high for the dataset and we see that annual mean precipitation and clay are signficant predictors of SOC, but there is also 
a significant period x clay and period x silt and period x precipitation effect. We need to explore what this means. 

For now, let's see if we can improve our model by controlling for country fixed effects. This could control for things such as how different countries may have different soil types and climates. 

In [ ]:
import pandas as pd
import pycountry
import pycountry_convert as pc

def get_continent(country_name):
    try:
        # Get the Alpha-2 country code
        country_code = pycountry.countries.lookup(country_name).alpha_2
        # Convert Alpha-2 country code to continent code
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        # Map continent codes to names
        continent_dict = {
            'AF': 'Africa',
            'AS': 'Asia',
            'EU': 'Europe',
            'NA': 'North America',
            'SA': 'South America',
            'OC': 'Oceania',
            'AN': 'Antarctica'
        }
        return continent_dict[continent_code]
    except:
        return 'Unknown'
    
# Example usage on your DataFrame
# Assuming your DataFrame is called combined_df with a 'country' column
combined_df['continent'] = combined_df['country'].apply(get_continent)

# Display the first few rows to verify
print(combined_df[['country', 'continent']].head())




  country      continent
0  Brazil  South America
1  Brazil  South America
2  Brazil  South America
3  Brazil  South America
4  Brazil  South America
       country      continent
0       Brazil  South America
27     Germany         Europe
72    Cameroon         Africa
146  Venezuela  South America
164    Ireland         Europe
180     France         Europe
186        USA  North America
238         UK        Unknown
248      Ghana         Africa
266      Spain         Europe
775      italy         Europe
791   Portugal         Europe


In [17]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

#First, let's use country as a fixed effect

# Assume 'combined_df' is your cleaned DataFrame with relevant variables
# Create a cateforical variable for 'continent'
combined_df['country'] = combined_df['country'].astype('category')

#Let's also try to account for the fixed effects of different climate types
# Convert 'climate' to a categorical variable
combined_df['climate'] = combined_df['climate'].astype('category')

# Define the regression formula with fixed effects (country) and other predictors
formula = 'SOC_Mg_ha ~ temperature + precipitation + clay + silt + sand + C(country) + depth_midpoint + C(climate)'

# Fit the model using Ordinary Least Squares (OLS)
model = smf.ols(formula=formula, data=combined_df).fit()

# Print the summary of the model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              SOC_Mg_ha   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.900
Method:                 Least Squares   F-statistic:                     291.2
Date:                Thu, 14 Nov 2024   Prob (F-statistic):          1.79e-285
Time:                        13:38:54   Log-Likelihood:                -2585.5
No. Observations:                 612   AIC:                             5211.
Df Residuals:                     592   BIC:                             5299.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [18]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Assume 'combined_df' is your cleaned DataFrame with relevant variables
# Create a cateforical variable for 'continent'
combined_df['continent'] = combined_df['continent'].astype('category')

#Let's also try to account for the fixed effects of different climate types
# Convert 'climate' to a categorical variable
combined_df['climate'] = combined_df['climate'].astype('category')

# Define the regression formula with fixed effects (country) and other predictors
formula = 'SOC_Mg_ha ~ temperature + precipitation + clay + silt + sand + C(continent) + depth_midpoint + C(climate)'

# Fit the model using Ordinary Least Squares (OLS)
model = smf.ols(formula=formula, data=combined_df).fit()

# Print the summary of the model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              SOC_Mg_ha   R-squared:                       0.679
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     97.38
Date:                Thu, 14 Nov 2024   Prob (F-statistic):          4.37e-138
Time:                        13:38:54   Log-Likelihood:                -2952.6
No. Observations:                 612   AIC:                             5933.
Df Residuals:                     598   BIC:                             5995.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [19]:
print(df_ph.head())

    ID  IDstudy  plotID     country         region  temperature_Celsiul  \
25  26        4       8  New_Zeland  Bay_of_Plenty                 14.5   
26  27        4       8  New_Zeland  Bay_of_Plenty                 14.5   
27  28        4       8  New_Zeland  Bay_of_Plenty                 14.5   
28  29        4       8  New_Zeland  Bay_of_Plenty                 14.5   
29  30        4       8  New_Zeland  Bay_of_Plenty                 14.5   

    precipitation_mm    climate  Latitud  Longitud  ... %clay_previous  \
25             138.0  Temperate  -37.616   175.916  ...            NaN   
26             138.0  Temperate  -37.616   175.916  ...            NaN   
27             138.0  Temperate  -37.616   175.916  ...            NaN   
28             138.0  Temperate  -37.616   175.916  ...            NaN   
29             138.0  Temperate  -37.616   175.916  ...            NaN   

   %silt_previous %sand_previous ph_previous SOC_g_kg_previous  \
25            NaN            NaN      

In [20]:
# Create separate dataframes for 'current' and 'previous' data
current_df_ph = df_ph[['ID', 'IDstudy', 'plotID', 'temperature_Celsiul', 'precipitation_mm', '%clay_current', '%silt_current', '%sand_current', 
                 'ph_current', 'SOC_Mg_ha_current', 'soil_from_cm_current', 'soil_to_cm_current', 'depth_midpoint_current']].copy()
previous_df_ph = df_ph[['ID', 'IDstudy', 'plotID', 'temperature_Celsiul', 'precipitation_mm', '%clay_previous', '%silt_previous', '%sand_previous', 
                  'ph_previous', 'SOC_Mg_ha_previous', 'soil_from_cm_previous', 'soil_to_cm_previous', 'depth_midpoint_previous']].copy()

# Rename columns for clarity
current_df_ph.columns = ['ID', 'IDstudy', 'plotID', 'temperature', 'precipitation', 'clay', 'silt', 'sand', 'pH', 'SOC_Mg_ha', 'soil_from_cm', 'soil_to_cm', 'depth_midpoint']
previous_df_ph.columns = ['ID', 'IDstudy', 'plotID', 'temperature', 'precipitation', 'clay', 'silt', 'sand', 'pH', 'SOC_Mg_ha', 'soil_from_cm', 'soil_to_cm', 'depth_midpoint']

# Add a column to indicate current vs previous
current_df_ph['Period'] = 'Current'
previous_df_ph['Period'] = 'Previous'

# Concatenate the dataframes
combined_df_ph = pd.concat([current_df_ph, previous_df_ph], ignore_index=True)

# Drop rows with NaN SOC_Mg_ha
combined_df_ph.dropna(subset=['SOC_Mg_ha'], inplace=True)

# Drop rows with NaN clay, silt, or sand content
combined_df_ph.dropna(subset=['clay', 'silt', 'sand'], inplace=True)

print(combined_df_ph.head())


    ID  IDstudy  plotID  temperature  precipitation  clay  silt  sand    pH  \
11  61        6      15          9.1          714.0  22.8  74.8   2.4  7.38   
12  62        6      16          9.1          714.0  22.8  74.8   2.4  7.38   
13  63        6      17          9.1          714.0  22.8  74.8   2.4  7.38   
14  64        6      15          9.1          714.0  22.8  74.8   2.4  7.38   
15  65        6      16          9.1          714.0  22.8  74.8   2.4  7.38   

    SOC_Mg_ha  soil_from_cm  soil_to_cm  depth_midpoint   Period  
11       16.0           0.0        10.0             5.0  Current  
12       16.1           0.0        10.0             5.0  Current  
13       18.4           0.0        10.0             5.0  Current  
14       13.4          10.0        20.0            15.0  Current  
15       13.4          10.0        20.0            15.0  Current  


In [21]:
#Let's repeat the same process but this time we will include only data for which we have pH values, and see if the model performs better


# Define your feature set (X) and target variable (y)
features = ['temperature', 'precipitation', 'clay', 'silt', 'sand', 'pH', 'depth_midpoint']
X = combined_df_ph[features]
y = combined_df_ph['SOC_Mg_ha']

# Encode the 'Period' variable and add interaction terms
combined_df_ph['Period'] = combined_df_ph['Period'].map({'Current': 1, 'Previous': 0})
for feature in features:
    combined_df_ph[f'{feature}_x_Period'] = combined_df_ph[feature] * combined_df_ph['Period']

# Update feature set to include interaction terms
interaction_features = features + [f'{feature}_x_Period' for feature in features]

X = combined_df_ph[interaction_features]

# Add a constant for the intercept
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              SOC_Mg_ha   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     14.30
Date:                Thu, 14 Nov 2024   Prob (F-statistic):           1.54e-26
Time:                        13:38:54   Log-Likelihood:                -1394.7
No. Observations:                 336   AIC:                             2819.
Df Residuals:                     321   BIC:                             2877.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

We can see from the results of this second regression that the model performs best when neglect pH because there are so many NaN values for pH, that
choosing to drop it as a variable allows us to keep more of our data for the regression. 

In [22]:
#Let's use our current and previous df dataframes to create a dataframe with delta values for each variable

# Create separate dataframes for 'current' and 'previous' data
current_df = df[['ID', 'IDstudy', 'plotID', 'country', 'region', 'climate', 'temperature_Celsiul', 'precipitation_mm', '%clay_current', '%silt_current', '%sand_current', 
                 'ph_current', 'SOC_Mg_ha_current', 'soil_from_cm_current', 'soil_to_cm_current', 'depth_midpoint_current']].copy()
previous_df = df[['ID', 'IDstudy', 'plotID', 'country', 'region', 'climate', 'temperature_Celsiul', 'precipitation_mm', '%clay_previous', '%silt_previous', '%sand_previous', 
                  'ph_previous', 'SOC_Mg_ha_previous', 'soil_from_cm_previous', 'soil_to_cm_previous', 'depth_midpoint_previous']].copy()

#Calculate a delta dataframe for each variable
delta_df = current_df.copy()

# Calculate the delta for each variable only where the variable changes between current and previous (ie. is labeled with _current and _previous)
for col in current_df.columns:
    if col.endswith('_current'):
        prev_col = col.replace('_current', '_previous')
        delta_df[col.replace('_current', '_delta')] = current_df[col] - previous_df[prev_col]

print(delta_df.head())

   ID  IDstudy  plotID country    region   climate  temperature_Celsiul  \
0   1        1       1  Brazil  SaoPaulo  Tropical                 21.0   
1   2        1       1  Brazil  SaoPaulo  Tropical                 21.0   
2   3        1       1  Brazil  SaoPaulo  Tropical                 21.0   
3   4        1       1  Brazil  SaoPaulo  Tropical                 21.0   
4   5        1       2  Brazil  SaoPaulo  Tropical                 21.0   

   precipitation_mm  %clay_current  %silt_current  ...  soil_to_cm_current  \
0            1500.0           13.7            5.1  ...                10.0   
1            1500.0           14.8            1.1  ...                20.0   
2            1500.0           18.0            2.0  ...                60.0   
3            1500.0           18.8            3.5  ...               100.0   
4            1500.0           38.1           10.9  ...                10.0   

   depth_midpoint_current  %clay_delta  %silt_delta  %sand_delta  ph_delta  \
0 

In [23]:
#Concatenate the delta dataframe with the luc_rank dataframe
print(luc_rank.head())

print(delta_df.head())

#Merge the two dataframes on the both the IDstudy and plotID columns
luc_combined = pd.merge(delta_df, luc_rank, on=['IDstudy', 'plotID'])

print(luc_combined.head())

#Rename land_use_change_rank2 to luc_rank
luc_combined = luc_combined.rename(columns={'land_use_change_rank2': 'luc_rank'})

#Rename the temperature_Celsiul column to temperature 
luc_combined = luc_combined.rename(columns={'temperature_Celsiul': 'temperature'})
luc_combined= luc_combined.rename(columns={'precipitation_mm': 'precipitation'})
luc_combined = luc_combined.rename(columns={'%clay_delta': 'delta_clay'})
luc_combined = luc_combined.rename(columns={'%silt_delta': 'delta_silt'})
luc_combined = luc_combined.rename(columns={'%sand_delta': 'delta_sand'})

   IDstudy  plotID  land_use_change_rank2
0        1       3                      4
1        1       3                      4
2        1       3                      4
3        1       3                      4
4        1       4                      2
   ID  IDstudy  plotID country    region   climate  temperature_Celsiul  \
0   1        1       1  Brazil  SaoPaulo  Tropical                 21.0   
1   2        1       1  Brazil  SaoPaulo  Tropical                 21.0   
2   3        1       1  Brazil  SaoPaulo  Tropical                 21.0   
3   4        1       1  Brazil  SaoPaulo  Tropical                 21.0   
4   5        1       2  Brazil  SaoPaulo  Tropical                 21.0   

   precipitation_mm  %clay_current  %silt_current  ...  soil_to_cm_current  \
0            1500.0           13.7            5.1  ...                10.0   
1            1500.0           14.8            1.1  ...                20.0   
2            1500.0           18.0            2.0  ...         

In [24]:
#Run an multiple regression model that predicts the change in SOC stock based on the change in land use and the change in the other variables

#Set luc_rank as a categorical variable
luc_combined['luc_rank'] = luc_combined['luc_rank'].astype('category')

#Set climate as a categorical variable
luc_combined['climate'] = luc_combined['climate'].astype('category')

#Drop all NaN values
luc_combined = luc_combined.dropna()

# Define the regression formula with fixed effects (country) and other predictors
formula = 'SOC_Mg_ha_delta ~ temperature + precipitation + C(luc_rank) + C(climate)'

# Fit the model using Ordinary Least Squares (OLS)
model = smf.ols(formula=formula, data=luc_combined).fit()

# Print the summary of the model

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:        SOC_Mg_ha_delta   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.224
Method:                 Least Squares   F-statistic:                     15.60
Date:                Thu, 14 Nov 2024   Prob (F-statistic):           8.66e-18
Time:                        13:38:54   Log-Likelihood:                -1145.1
No. Observations:                 355   AIC:                             2306.
Df Residuals:                     347   BIC:                             2337.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         